<a href="https://colab.research.google.com/github/Regis0323/Module_1/blob/main/Rurangwa_Assignment_Module_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="background-color: #004B87; padding: 20px; text-align: center; border-radius: 10px; color: white; font-family: Arial, sans-serif; margin: auto; width: 80%;">
    <h1>IABE Data Science Certificate Module 1 Assignment</h1>
    <h2>RURANGWA IRADUKUNDA Jean-François Régis</h2>
    <h3>31st January, 2025</h3>
    <p>Email : jeanfrancoisregis.rurangwairadukunda@axa.be</p>
</div>


## Table of contents
1. [Introduction](#Introduction)<br />
2. [Data Exploration and Preprocessing](#Data_treatment)<br />
3. [Linear Models and Conditional Expectation Estimation](#Linear_Models)<br />
4. [Generalized Linear Models (GLM)](#GLM)<br />
  4.1. [Data prep](#Data_prep)<br />
  4.2. [Ridge Regression](#Ridge)<br />
  4.3. [Lasso Regression](#Lasso)<br />
  4.4. [ElasticNet Regression](#ElasticNet)<br />
  4.5. [Support Vector Machines](#SVM)<br />
  4.6. [Model Comparison and Visaulizations](#Model_Comparison)<br />
5. [Conclusion](#Conclusion)<br />

# 1. Introduction<a name="Introduction"></a>


Under the 'Actuarial Data Scientist' program given by the Belgian association of actuaries (IABE), an assignment was given to us after the first (out of three) module to help us review all the concepts via a practical case.
<br/>
The assignment consists in analysing the "**Auto Insurance Claims Data**" dataset (retrieved on Kaggle) using the concepts learned in the module. This dataset contains different features related to auto insurance claims and tells if a claim was fraudulent or not.
<br/>
We shall now start the process which will lead us to determining the best predictive model that predicts if an insurance claim is fraudulent or not, starting by the data importation and processing, followed by the implementation of different models and ending up with a comparison of the different models before choosing the best one.

#2. Data Exploration and Preprocessing<a name="Data_treatment"></a>

We start by importing different packages that will be of use throughout the analysis.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.ticker import MaxNLocator
import plotly.graph_objs as go
import plotly
import time
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut
!git clone https://github.com/Regis0323/Import_files.git
!pip install scikit-fuzzy

# Packages used for PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
from sklearn.metrics import r2_score

# KMeans
from scipy.spatial.distance import euclidean
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples

# MiniBatchKMeans
from sklearn.cluster import MiniBatchKMeans

# Fuzzy clustering
import skfuzzy as fuzz

# Spectral Clustering
from sklearn.cluster import SpectralClustering

Then we can proceed importing the dataset and having a small look of the data alongside the different types of the variables.

In [65]:
claims_data = pd.read_csv("/content/Import_files/insurance_claims.csv")
display(claims_data.head())
print(claims_data.dtypes.value_counts())

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


object     21
int64      17
float64     2
Name: count, dtype: int64


Our dataset is composed of 40 columns, of which 21 are categorized as 'object', 17 as 64-bit 'integer' (int64) and 2 as 64-bit 'floating-point' numbers.
<br>
After a quick look of the 5 first rows, we can already see that some lines contain missing values ('?' and 'NaN'). We shall now do a thorough test to identify those missing values and treat them accordingly

In [66]:
claims_data.info()
claims_data['authorities_contacted'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   months_as_customer           1000 non-null   int64  
 1   age                          1000 non-null   int64  
 2   policy_number                1000 non-null   int64  
 3   policy_bind_date             1000 non-null   object 
 4   policy_state                 1000 non-null   object 
 5   policy_csl                   1000 non-null   object 
 6   policy_deductable            1000 non-null   int64  
 7   policy_annual_premium        1000 non-null   float64
 8   umbrella_limit               1000 non-null   int64  
 9   insured_zip                  1000 non-null   int64  
 10  insured_sex                  1000 non-null   object 
 11  insured_education_level      1000 non-null   object 
 12  insured_occupation           1000 non-null   object 
 13  insured_hobbies    

,count
authorities_contacted,
Police,292
Fire,223
Other,198
Ambulance,196


After this first check, we can see that the last column '_c39' is empty and can be deleted. Furthermore, it is shown that the variable 'authorities_contracted' has 91 missing values, but, after a closer look into the variable, those 91 values refer to the cases where **NO** authorities were called/contacted. This misunderstanding came from the fact that there were referred to as 'None' so we shall replace that and put 'Zero' instead.

In [67]:
claims_data.drop('_c39',axis=1,inplace=True)
claims_data.fillna(value='Zero', inplace=True)
claims_data.info()
claims_data['authorities_contacted'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   months_as_customer           1000 non-null   int64  
 1   age                          1000 non-null   int64  
 2   policy_number                1000 non-null   int64  
 3   policy_bind_date             1000 non-null   object 
 4   policy_state                 1000 non-null   object 
 5   policy_csl                   1000 non-null   object 
 6   policy_deductable            1000 non-null   int64  
 7   policy_annual_premium        1000 non-null   float64
 8   umbrella_limit               1000 non-null   int64  
 9   insured_zip                  1000 non-null   int64  
 10  insured_sex                  1000 non-null   object 
 11  insured_education_level      1000 non-null   object 
 12  insured_occupation           1000 non-null   object 
 13  insured_hobbies    

,count
authorities_contacted,
Police,292
Fire,223
Other,198
Ambulance,196
Zero,91


As for the variables with '?' in their values, we shall identify them and replace them with the mode or the mean of the specified variable so that we won't encounter any 'missing values' problem, found when using some packages, while doing our study.

In [68]:
lookup_value = '?'
output = claims_data.isin([lookup_value])
values=claims_data.loc[:,output.any(axis=0)]
print(values)

      collision_type property_damage police_report_available
0     Side Collision             YES                     YES
1                  ?               ?                       ?
2     Rear Collision              NO                      NO
3    Front Collision               ?                      NO
4                  ?              NO                      NO
..               ...             ...                     ...
995  Front Collision             YES                       ?
996   Rear Collision             YES                       ?
997   Side Collision               ?                     YES
998   Rear Collision               ?                     YES
999                ?               ?                       ?

[1000 rows x 3 columns]


Three variables are identified in this case: 'collision_type','property_damage'and 'police_report_available.

In [69]:
#Before the replacement
print(claims_data['collision_type'].value_counts())
print(claims_data['property_damage'].value_counts())
print(claims_data['police_report_available'].value_counts())

collision_type
Rear Collision     292
Side Collision     276
Front Collision    254
?                  178
Name: count, dtype: int64
property_damage
?      360
NO     338
YES    302
Name: count, dtype: int64
police_report_available
?      343
NO     343
YES    314
Name: count, dtype: int64


It is best to replace the values found in the 'collision_type' variable with its mode (='Rear Collision') and the value 'NO' for the two remaining variables as its their second most frequent value after '?' which is the mode for both of them.

In [71]:
#After the replacement
claims_data['collision_type'].replace(['?'], claims_data['collision_type'].mode([0]), inplace=True)
claims_data['property_damage'].replace(['?'], ['NO'], inplace=True)
claims_data['police_report_available'].replace(['?'], ['NO'], inplace=True)
print(claims_data['collision_type'].value_counts())
print(claims_data['property_damage'].value_counts())
print(claims_data['police_report_available'].value_counts())

collision_type
Rear Collision     470
Side Collision     276
Front Collision    254
Name: count, dtype: int64
property_damage
NO     698
YES    302
Name: count, dtype: int64
police_report_available
NO     686
YES    314
Name: count, dtype: int64


In [ ]:
display(claims_data.describe((include=['object', 'float', 'int'])))
display(claims_data.head())

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000
mean,203.954000,38.948000,546238.648000,1136.000000,1256.406150,1.101000e+06,501214.488000,25126.100000,-26793.700000,11.644000,1.83900,0.992000,1.487000,52761.94000,7433.420000,7399.570000,37928.950000,2005.103000
std,115.113174,9.140287,257063.005276,611.864673,244.167395,2.297407e+06,71701.610941,27872.187708,28104.096686,6.951373,1.01888,0.820127,1.111335,26401.53319,4880.951853,4824.726179,18886.252893,6.015861
min,0.000000,19.000000,100804.000000,500.000000,433.330000,-1.000000e+06,430104.000000,0.000000,-111100.000000,0.000000,1.00000,0.000000,0.000000,100.00000,0.000000,0.000000,70.000000,1995.000000
25%,115.750000,32.000000,335980.250000,500.000000,1089.607500,0.000000e+00,448404.500000,0.000000,-51500.000000,6.000000,1.00000,0.000000,1.000000,41812.50000,4295.000000,4445.000000,30292.500000,2000.000000
50%,199.500000,38.000000,533135.000000,1000.000000,1257.200000,0.000000e+00,466445.500000,0.000000,-23250.000000,12.000000,1.00000,1.000000,1.000000,58055.00000,6775.000000,6750.000000,42100.000000,2005.000000
75%,276.250000,44.000000,759099.750000,2000.000000,1415.695000,0.000000e+00,603251.000000,51025.000000,0.000000,17.000000,3.00000,2.000000,2.000000,70592.50000,11305.000000,10885.000000,50822.500000,2010.000000
max,479.000000,64.000000,999435.000000,2000.000000,2047.590000,1.000000e+07,620962.000000,100500.000000,0.000000,23.000000,4.00000,2.000000,3.000000,114920.00000,21450.000000,23670.000000,79560.000000,2015.000000


,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,...,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,...,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,...,0,?,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,...,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,...,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,...,1,NO,6500,1300,650,4550,Accura,RSX,2009,N


In [ ]:
# General information on the dataset
print(claims_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 39 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   months_as_customer           1000 non-null   int64  
 1   age                          1000 non-null   int64  
 2   policy_number                1000 non-null   int64  
 3   policy_bind_date             1000 non-null   object 
 4   policy_state                 1000 non-null   object 
 5   policy_csl                   1000 non-null   object 
 6   policy_deductable            1000 non-null   int64  
 7   policy_annual_premium        1000 non-null   float64
 8   umbrella_limit               1000 non-null   int64  
 9   insured_zip                  1000 non-null   int64  
 10  insured_sex                  1000 non-null   object 
 11  insured_education_level      1000 non-null   object 
 12  insured_occupation           1000 non-null   object 
 13  insured_hobbies    

In [ ]:
nominal=[]
discrete = []
continuous = []
for i in list(claims_data):
  if claims_data[i].dtype == 'object':
    nominal.append(i)
  elif claims_data[i].dtype == 'float64':
    continuous.append(i)
  else:
      discrete.append(i)
print("Nominal variables:")
print("------------------")
print(nominal)
print("\nDiscrete variables:")
print("-------------------")
print(discrete)
print("\nContinuous variables:")
print("---------------------")
print(continuous)

Nominal variables:
------------------
['policy_bind_date', 'policy_state', 'policy_csl', 'insured_sex', 'insured_education_level', 'insured_occupation', 'insured_hobbies', 'insured_relationship', 'incident_date', 'incident_type', 'collision_type', 'incident_severity', 'authorities_contacted', 'incident_state', 'incident_city', 'incident_location', 'property_damage', 'police_report_available', 'auto_make', 'auto_model', 'fraud_reported']

Discrete variables:
-------------------
['months_as_customer', 'age', 'policy_number', 'policy_deductable', 'umbrella_limit', 'insured_zip', 'capital-gains', 'capital-loss', 'incident_hour_of_the_day', 'number_of_vehicles_involved', 'bodily_injuries', 'witnesses', 'total_claim_amount', 'injury_claim', 'property_claim', 'vehicle_claim', 'auto_year']

Continuous variables:
---------------------
['policy_annual_premium', '_c39']


In [ ]:
claims_data.describe(include=['object', 'float', 'int'])

,months_as_customer,age,policy_number,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,capital-gains,capital-loss,incident_hour_of_the_day,number_of_vehicles_involved,bodily_injuries,witnesses,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_year,_c39
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,0.0
mean,203.954000,38.948000,546238.648000,1136.000000,1256.406150,1.101000e+06,501214.488000,25126.100000,-26793.700000,11.644000,1.83900,0.992000,1.487000,52761.94000,7433.420000,7399.570000,37928.950000,2005.103000,NaN
std,115.113174,9.140287,257063.005276,611.864673,244.167395,2.297407e+06,71701.610941,27872.187708,28104.096686,6.951373,1.01888,0.820127,1.111335,26401.53319,4880.951853,4824.726179,18886.252893,6.015861,NaN
min,0.000000,19.000000,100804.000000,500.000000,433.330000,-1.000000e+06,430104.000000,0.000000,-111100.000000,0.000000,1.00000,0.000000,0.000000,100.00000,0.000000,0.000000,70.000000,1995.000000,NaN
25%,115.750000,32.000000,335980.250000,500.000000,1089.607500,0.000000e+00,448404.500000,0.000000,-51500.000000,6.000000,1.00000,0.000000,1.000000,41812.50000,4295.000000,4445.000000,30292.500000,2000.000000,NaN
50%,199.500000,38.000000,533135.000000,1000.000000,1257.200000,0.000000e+00,466445.500000,0.000000,-23250.000000,12.000000,1.00000,1.000000,1.000000,58055.00000,6775.000000,6750.000000,42100.000000,2005.000000,NaN
75%,276.250000,44.000000,759099.750000,2000.000000,1415.695000,0.000000e+00,603251.000000,51025.000000,0.000000,17.000000,3.00000,2.000000,2.000000,70592.50000,11305.000000,10885.000000,50822.500000,2010.000000,NaN
max,479.000000,64.000000,999435.000000,2000.000000,2047.590000,1.000000e+07,620962.000000,100500.000000,0.000000,23.000000,4.00000,2.000000,3.000000,114920.00000,21450.000000,23670.000000,79560.000000,2015.000000,NaN


In [ ]:
import plotly
pd.options.plotting.backend = "matplotlib"

# 3. Linear Models and Conditional Expectation Estimation<a name="Linear_Models"></a>

# 4. Generalized Linear Models (GLM)<a name="GLM"></a>

## 4.1. Data prep<a name="Data_prep"></a>

## 4.2. Ridge Regression<a name="Ridge"></a>

## 4.3. Lasso Regression<a name="Lasso"></a>

## 4.4. ElasticNet Regression<a name="ElasticNet"></a>

## 4.5. Support Vector Machines<a name="SVM"></a>

## 4.6. Model Comparison and Visaulizations<a name="Model_Comparison"></a>

# 5. Conclusion<a name="Conclusion"></a>